## Mathematics of Backpropagation

Remember that our ultimate goal in training a neural network is to find the gradient of each weight with respect to the output:

$$\begin{align}
  \frac{\partial E}{\partial w_{i\rightarrow j}}
\end{align}$$

We do this so that we can update the weights incrementally using stochastic gradient descent: 

$$\begin{align*}
  w_{i\rightarrow j} =& w_{i\rightarrow j} -\eta \frac{\partial E}{\partial
    w_{i\rightarrow j}}
\end{align*}$$


For a single unit in a general network, we can have several cases: the unit may have only one input and one output (case 1), the unit may have multiple inputs (case 2), or the unit may have multiple outputs (case 3). Technically there is a fourth case: a unit may have multiple inputs and outputs. But as we will see, the multiple input case and the multiple output case are independent, and we can simply combine the rules we learn for case 2 and case 3 for this case.

**Case 1: Single input and single output**

Suppose we have the following network:

![](https://images.squarespace-cdn.com/content/v1/51d342a0e4b0290bcc56387d/1414538372467-FSROEKYUMKZZOO8I43HW/ke17ZwdGBToddI8pDm48kMLc0aLpIXxdLiU6DmD0NspZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxxJSzaL75CZMWUQE-S_-b5OKDW-NZ7BDgc1ZWJrOepjj7htahwPQ9PmwcyqKUpcuM/image-asset.png?format=750w)

We can explicitly write out the values of each of variable in this network:

$$
\begin{align}
  s_j =&\ w_1\cdot x_i\\
  z_j =&\ \sigma(in_j) = \sigma(w_1\cdot x_i)\\
  s_k =&\ w_2\cdot z_j\\
  z_k =&\ \sigma(in_k) = \sigma(w_2\cdot\sigma(w_1\cdot x_i))\\
  s_o =&\ w_3\cdot z_k\\
  \hat{y}_i =&\ in_o = w_3\cdot\sigma(w_2\cdot\sigma(w_1\cdot x_i))\\
  E =&\ \frac{1}{2}(\hat{y}_i - y_i)^2 =
  \frac{1}{2}(w_3\cdot\sigma(w_2\cdot\sigma(w_1\cdot x_i)) - y_i)^2
\end{align}
$$


First, let's find the derivative for $w_{k\rightarrow o}$ (remember that $\hat{y} = w_{k\rightarrow o}z_k$, as our output is a linear unit):

$$
\begin{align}
  \frac{\partial E}{\partial w_{k\rightarrow o}} =&\ \frac{\partial}{\partial w_{k\rightarrow o}}
  \frac{1}{2}(\hat{y}_i - y_i)^2\\
  =&\ \frac{\partial}{\partial w_{k\rightarrow o}} \frac{1}{2}(w_{k\rightarrow o}\cdot z_k - y_i)^2\\
  =&\ (w_{k\rightarrow o}\cdot z_k - y_i)\frac{\partial}{\partial w_{k\rightarrow o}}(w_{k\rightarrow o}\cdot z_k -
  y_i)\\
  =&\ \color{blue}{(\hat{y_i} - y_i)}(z_k)
\end{align}
$$

Finding the weight update for $w_{i\rightarrow k}$ is also relatively simple:

$$
\begin{align}
  \frac{\partial E}{\partial w_{j\rightarrow k}} =&\ \frac{\partial}{\partial w_{j\rightarrow k}}
  \frac{1}{2}(\hat{y}_i - y_i)^2\\
 =&\ (\hat{y}_i-y_i)\left( \frac{\partial}{\partial w_{j\rightarrow k}} (w_{k\rightarrow o}\cdot\sigma(w_{j\rightarrow k}\cdot
  z_j) - y_i) \right)\\
  =&\ (\hat{y}_i-y_i)(w_{k\rightarrow o})\left( \frac{\partial}{\partial w_{j\rightarrow k}} \sigma(w_{j\rightarrow k}\cdot
  z_j) \right)\\
  =&\ (\hat{y}_i-y_i)(w_{k\rightarrow o})\left( \sigma(s_k)(1-\sigma(s_k)) \frac{\partial
 }{\partial w_{j\rightarrow k}}(w_{j\rightarrow k}\cdot z_j) \right)\\
 =&\ \color{blue}{(\hat{y}_i-y_i)}\color{red}{(w_{k\rightarrow o})\left(
 \sigma(s_k)(1-\sigma(s_k)\right)}(z_j)
\end{align}
$$

Again, finding the weight update for $w_{i\rightarrow j}$ consists of some straightforward calculus:

$$
\begin{align}
  \frac{\partial E}{\partial w_{i\rightarrow j}} =&\ \frac{\partial}{\partial w_{i\rightarrow j}}
  \frac{1}{2}(\hat{y}_i-y_i)^2\\
  =&\ (\hat{y}_i-y_i)\left( \frac{\partial}{\partial w_{i\rightarrow j}} (\hat{y}_i-y_i)
 \right)\\
  =&\ (\hat{y}_i-y_i)(w_{k\rightarrow o})\left( \frac{\partial}{\partial
  w_{i\rightarrow j}}\cdot\sigma(w_{j\rightarrow k}\cdot\sigma(w_{i\rightarrow j}\cdot x_i))\right)\\
  =&\ (\hat{y}_i-y_i)(w_{k\rightarrow o})(\sigma(s_k)(1-\sigma(s_k)))(w_{j\rightarrow k})\left(
  \frac{\partial}{\partial w_{i\rightarrow j}}\sigma(w_{i\rightarrow j}\cdot x_i) \right)\\
  =&\
  \color{blue}{(\hat{y}_i-y_i)}\color{red}{(w_{k\rightarrow o})(\sigma(s_k)(1-\sigma(s_k)))}\color{OliveGreen}{(w_{j\rightarrow k})(\sigma(s_j)(1-\sigma(s_j)))}(x_i)
\end{align}
$$

By now, you should be seeing a pattern emerging, a pattern that hopefully we could encode with backpropagation. We are reusing multiple values as we compute the updates for weights that appear earlier and earlier in the network. Specifically, we see the derivative of the network error, the weighted derivative of unit k's output with respect to sk, and the weighted derivative of unit j's output with respect to sj.

So, in summary, for this simple network, we have:

$$
\begin{align}
  \Delta w_{i\rightarrow j} =&\ -\eta\left[
  \color{blue}{(\hat{y}_i-y_i)}\color{red}{(w_{k\rightarrow o})(\sigma(s_k)(1-\sigma(s_k)))}\color{OliveGreen}{(w_{j\rightarrow k})(\sigma(s_j)(1-\sigma(s_j)))}(x_i)
  \right]\\
  \Delta w_{j\rightarrow k} =&\ -\eta\left[
  \color{blue}{(\hat{y}_i-y_i)}\color{red}{(w_{k\rightarrow o})\left(
  \sigma(s_k)(1-\sigma(s_k)\right)}(z_j)\right]\\
  \Delta w_{k\rightarrow o} =&\ -\eta\left[ \color{blue}{(\hat{y_i} - y_i)}(z_k)\right]
\end{align}
$$

**Case 2: Handling multiple inputs**

Consider the more complicated network, where a unit may have more than one input:

![](https://images.squarespace-cdn.com/content/v1/51d342a0e4b0290bcc56387d/1414538430117-013J6OBJRLNJORGFCEUD/ke17ZwdGBToddI8pDm48kNGhvwK1dHyJEVTMYsDQT29Zw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpyuNHSz5nMm54PQVFA1hE7KfIV1CslcfaIkUKLdPzeHlsm0QOROT0AAQiev9IOwSsI/image-asset.png?format=750w)

What happens to a weight when it leads to a unit that has multiple inputs? Is $w_{i\rightarrow k}$'s update rule affected by $w_{j\rightarrow k}$'s update rule? To see, let's derive the update for $w_{i\rightarrow k}$ by hand:
$$
\begin{align}
  \frac{\partial E}{w_{i\rightarrow k}} =& \frac{\partial}{w_{i\rightarrow
  k}}\frac{1}{2}(\hat{y}_i - y_i)^2\\
  =&\ (\hat{y}_i - y_i)\left( \frac{\partial}{w_{i\rightarrow k}}z_k w_{k\rightarrow
  o} \right)\\
  =&\ (\hat{y}_i - y_i)(w_{k\rightarrow o})\left(
  \frac{\partial}{w_{i\rightarrow k}}\sigma\left( s_k \right) \right)\\
  =&\ (\hat{y}_i - y_i)(\sigma(s_k)(1-\sigma(s_k)) w_{k\rightarrow o})\left(
  \frac{\partial}{w_{i\rightarrow k}}\left( z_iw_{i\rightarrow k} +
  z_jw_{j\rightarrow k} \right) \right)\\
  =&\ (\hat{y}_i - y_i)(\sigma(s_k)(1-\sigma(s_k)) w_{k\rightarrow o})z_i
\end{align}
$$

Here we see that the update for $w_{i\rightarrow k}$ does not depend on $w_{j\rightarrow k}$'s derivative, leading to our first rule: <i>The derivative for a weight is not dependent on the derivatives of any of the other weights in the same layer</i>. Thus we can update weights in the same layer in isolation. There is a natural ordering of the updates - they only depend on the <i>values</i> of other weights in the same layer, and (as we shall see), the derivatives of weights further in the network. This ordering is good news for the backpropagation algorithm.

**Case 3: Handling multiple outputs**

Now let's examine the case where a hidden unit has more than one output.

![](https://images.squarespace-cdn.com/content/v1/51d342a0e4b0290bcc56387d/1414538443764-IQKUIK5RPH2RGAEFR022/ke17ZwdGBToddI8pDm48kHgH4eLVOVXymExgxje_xptZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxMaBvyVKnGaRrghS_oMeqIXXJ2xUVo_MEPAEtd3K-iXQ6k8WxnFm6pcqD4HABN_rs/image-asset.png?format=750w)

Based on the previous sections, the only "new" type of weight update is the derivative of $w_{in\rightarrow j}$. The difference in the multiple output case is that unit $i$ has more than one immediate successor, so (spoiler!) we must sum the error accumulated along all paths that are rooted at unit $i$. Let's explicitly derive the weight update for $w_{in\rightarrow i}$ (to keep track of what's going on, we define $\sigma_i(\cdot)$ as the activation function for unit $i$): 
$$
\begin{align}
  \frac{\partial E}{w_{in\rightarrow i}} =& \frac{\partial}{w_{in\rightarrow
  i}}\frac{1}{2}(\hat{y}_i - y_i)^2\\
  =&\ (\hat{y}_i - y_i)\left( \frac{\partial}{w_{in\rightarrow i}}(z_j
    w_{j\rightarrow o} + z_k w_{k\rightarrow o}) \right)\\
  =&\ (\hat{y}_i - y_i)\left( \frac{\partial}{w_{in\rightarrow i}}(\sigma_j(s_j)
    w_{j\rightarrow o} + \sigma_k(s_k)w_{k\rightarrow o}) \right)\\
  =&\ (\hat{y}_i - y_i)\left( w_{j\rightarrow
    o}\sigma_j'(s_j) \frac{\partial}{w_{in\rightarrow i}}s_j  +
    w_{k\rightarrow o}\sigma_k'(s_k) \frac{\partial}{w_{in\rightarrow
    i}}s_k \right)\\
  =&\ (\hat{y}_i - y_i)\left( w_{j\rightarrow o}\sigma_j'(s_j)
    \frac{\partial}{w_{in\rightarrow i}}z_iw_{i\rightarrow j}  + w_{k\rightarrow
    o}\sigma_k'(s_k) \frac{\partial}{w_{in\rightarrow i}}z_iw_{i\rightarrow k}
    \right)\\
  =&\ (\hat{y}_i - y_i)\left( w_{j\rightarrow o}\sigma_j'(s_j)
    \frac{\partial}{w_{in\rightarrow i}}\sigma_i(s_i)w_{i\rightarrow j}  + w_{k\rightarrow
    o}\sigma_k'(s_k) \frac{\partial}{w_{in\rightarrow i}}\sigma_i(s_i)w_{i\rightarrow k}
    \right)\\
  =&\ (\hat{y}_i - y_i)\left( w_{j\rightarrow o}\sigma_j'(s_j)
    w_{i\rightarrow j}\sigma'_i(s_i)\frac{\partial}{w_{in\rightarrow i}}s_i +
    w_{k\rightarrow o}\sigma_k'(s_k) w_{i\rightarrow k}\sigma'_i(s_i)
    \frac{\partial}{w_{in\rightarrow i}}s_i
    \right)\\
  =&\ (\hat{y}_i - y_i)\left( w_{j\rightarrow o}\sigma_j'(s_j)
    w_{i\rightarrow j}\sigma'_i(s_i) + w_{k\rightarrow o}\sigma_k'(s_k)
    w_{i\rightarrow k}\sigma'_i(s_i) \right)x_i
\end{align}
$$

There are two things to note here. The first, and most relevant, is our second derived rule: the weight update for a weight leading to a unit with multiple outputs is dependent on derivatives that reside on both paths.